# 多通道输入和输出
- 一般一张图片是有多个通道的，比如所RGB就有三个通道
- 通道只是一种泛称、不要仅仅指颜色通道，它重要的意义是可以表示多个模式
- 比如说RGB三个通道，可以表示红、绿、蓝三种颜色，就要做三个通道
- 输入通道最后的矩阵叉乘起来，最后输出通道只会有一个
- 要得到多个输出通道，就要在输入通道上再做重复操作了
- 多个输入和输出最重要的是在中间的隐藏层来使用的

## 多个输入通道
- 输入X: $c_i \times n_h \times n_w $
- 核W: $c_i \times k_h \times k_w $
- 偏差B：$c_i $
- 输出Y: $m_h \times m_w $
- 公式表达： $Y = \sum_{i=1}^{c_i}X_i \cdot W_i + B$
- 备注：无论有多少个输入通道，最后会做矩阵的加法，最后就是一个输出通道

## 多个输入和输出通道
- 输入X: $c_i \times n_h \times n_w $
- 核W: $c_o \times c_i \times k_h \times k_w $
- 偏差B：$c_o \times c_i $
- 输出Y: $c_o \times m_h \times m_w $
- 公式表达： $Y_{i,:,:} = X \cdot W_{i,:,:,;} + B$, for i = 1, ..., c_o

## 总结
- 输入通道和输出通道是模型的超参数
- 每个输入通道都有一个独立的二维卷积核，所有的结果相加得到一个输出通道
- 所以每个输出通道都有一个独立的三维卷积核
- 所以最后的结果是一个四阶张量

In [1]:
# 实现多输入通道的相关运算
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [5]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
                  [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [6]:
# 实现多输出通道的相关运算
def corr2d_multi_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
K = torch.stack([K, K + 1, K + 2], 0)
corr2d_multi_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])